In [ ]:
from arcgis.gis import GIS
import pandas
import json
import getpass

# Make the connection to your ArcGIS Online Organization
orgURL = "https://virginiatech.maps.arcgis.com" # <==update for your org
orgUser = "sspeery"                             # <==update for your org
orgPwd = getpass.getpass("Enter your AGOL Administrator Password")
gis = GIS(orgURL,username=orgUser,password=orgPwd) 


# Retrieve all the entitlements in the org
lics = gis.admin.license.all()
licensedProducts = []   # array of the specific Products for which there may be entitlements
licenseArr = []         # array of the actual licenses for the Products
for lic in lics:
    try:
        licensedProducts.append(json.loads(str(lic.properties))['listing']['title'])
    except KeyError:
        pass

entDf = pandas.DataFrame()  # this stores your organization's entitlements total/assigned/remaining for each product in a dataframe
for product in licensedProducts:
    try:
        theLicense = gis.admin.license.get(product)
        licenseArr.append(theLicense)
        thisProductReportDf = theLicense.report #gets the total/assigned/remaining licenses for each product
        thisProductReportDf['Product'] = product
        entDf = entDf.append(thisProductReportDf)
    except KeyError:
        pass
    
# Create a dictionary of the form used by the UMich license assignment script
# See also: https://gist.github.com/sspeery/190d6dc45bc8f4c4b3ba0e2705850463
entitlements = dict.fromkeys(entDf['Product'].unique())
for key in entitlements:
    entitlements[key]=[]
for index, row in entDf.iterrows():
    entitlements[row['Product']].append(row['Entitlement'])
#entitlements
entDf